In [1]:
from quchem.Hamiltonian_Generator_Functions import *
from quchem.Graph import *
### HAMILTONIAN start
Molecule = 'H2O'
geometry = None # [('H', (0., 0., 0.)), ('H', (0., 0., 0.74))]
basis = 'sto-6g'


### Get Hamiltonian
Hamilt = Hamiltonian_PySCF(Molecule,
                     run_scf=1, run_mp2=1, run_cisd=1, run_ccsd=1, run_fci=1,
                     basis=basis,
                     multiplicity=1,
                     geometry=geometry)  # normally None!
QubitHamiltonian = Hamilt.Get_Qubit_Hamiltonian(threshold=None, transformation='BK')
### HAMILTONIAN end
QubitHamiltonian

(-46.893686761584505+0j) [] +
(0.10688883191080306+0j) [X0 X1 X2] +
(0.0018823069706634735+0j) [X0 X1 X2 X3 Y7 Y11] +
(0.0024331089238630566+0j) [X0 X1 X2 X3 Z7] +
(-0.003543941201579209+0j) [X0 X1 X2 Z3] +
(-0.0014795610736064787+0j) [X0 X1 Z2 X3 Y7 Z9 Y10 X11] +
(-0.0015611859902872525+0j) [X0 X1 Z2 Y3 Z5 Y6] +
(-0.0010022198919638747+0j) [X0 X1 X3 X4 Y5 Y6 Y7 Z11 Y12 X13] +
(-0.0007985318024164989+0j) [X0 X1 X3 X4 Y7 Y11] +
(-0.000851323751960051+0j) [X0 X1 X3 X4 Z7] +
(-0.0011507900568467453+0j) [X0 X1 X3 Y4 Y5 X6 Y7 Z11 Y12 X13] +
(0.0021530099488106203+0j) [X0 X1 X3 Y4 Y5 Y6 Y7 Z11 X12 X13] +
(-0.0023812815348219947+0j) [X0 X1 X3 Y4 Y5 Z6 Y7 Z11 Y13] +
(0.0007985318024164989+0j) [X0 X1 X3 Y4 Z5 Y7 Z9 Z10 X11] +
(0.0002282715860113743+0j) [X0 X1 X3 Z4 Y5 Y6 Y7 Z11 Y13] +
(-9.109928253980847e-05+0j) [X0 X1 X3 Z4 Z5 Y7 Z9 Y10 X11] +
(0.0007074325198766904+0j) [X0 X1 X3 Z4 Y7 Z9 Y10 X11] +
(0.0002282715860113743+0j) [X0 X1 X3 X5 Y6 Y7 Z11 Z12 X13] +
(-0.001230491477975249+0j) [X0 X1 

In [2]:
Hamilt.Get_HF_spin_orbital_occupation()

array([2., 2., 2., 2., 2., 0., 0.])

The H2O ground state under JW is:

$$|\psi \rangle_{HF}^{ground} = | 1 1 1 1 1 1 1 1 1 1 \:  0 0 0 0 \rangle$$

and under BK is:

$$|\psi \rangle_{HF}^{ground} = | 1 0 1 0 1 0 1 0 1 0 0 0 0 0 \rangle$$

In [3]:
from quchem.Ansatz_Generator_Functions import *
ansatz_obj = Ansatz(Hamilt.molecule.n_electrons,
                    Hamilt.molecule.n_qubits)

print(ansatz_obj.Get_JW_HF_state_in_OCC_basis())
print(ansatz_obj.Get_BK_HF_state_in_OCC_basis())

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
[1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0.]


Use NOON to reduces this:

In [4]:
from quchem.Ansatz_Generator_Functions import *

##
NOON_spins_combined, NMO_basis = Hamilt.Get_NOON()
##
Hamilt.Get_CCSD_Amplitudes()

NOON_spins_combined

array([0.02786304, 0.02830013, 1.97540204, 1.99999764, 1.99791916,
       1.97220576, 1.99831224])

In [5]:
# automate:
removed_indices, reduced_Sec_Quant_CC_ops_ia, reduced_Sec_Quant_CC_ops_ijab, reduced_theta_parameters_ia, reduced_theta_parameters_ijab =ansatz_obj.Remove_NOON_terms(
    NOON=NOON_spins_combined,
    occ_threshold= 1.998,
    unocc_threshold=1e-4,
    indices_to_remove_list_manual=None, 
    single_cc_amplitudes=Hamilt.molecule.single_cc_amplitudes,
    double_cc_amplitudes=Hamilt.molecule.double_cc_amplitudes,
    singles_hamiltonian=Hamilt.singles_hamiltonian,
    doubles_hamiltonian=Hamilt.doubles_hamiltonian,
    tol_filter_small_terms=None)

# # manual
# reduced_Sec_Quant_CC_ops_ia, reduced_Sec_Quant_CC_ops_ijab, reduced_theta_parameters_ia, reduced_theta_parameters_ijab =ansatz_obj.Remove_NOON_terms(
#     NOON=NOON_spins_combined,
#     indices_to_remove_list_manual=[0,1,8,9,10,11])

In [6]:
print(print(np.around(np.diag(NOON_spins_combined), 5)))
print('removed_indices =', removed_indices)

[[0.02786 0.      0.      0.      0.      0.      0.     ]
 [0.      0.0283  0.      0.      0.      0.      0.     ]
 [0.      0.      1.9754  0.      0.      0.      0.     ]
 [0.      0.      0.      2.      0.      0.      0.     ]
 [0.      0.      0.      0.      1.99792 0.      0.     ]
 [0.      0.      0.      0.      0.      1.97221 0.     ]
 [0.      0.      0.      0.      0.      0.      1.99831]]
None
removed_indices = [12, 13, 6, 7]


LOOK at PHYS. REV. X, **8**, 031022 (2018)

Due to NOON:
- indices 0,1 assumed to ALWAYS be filled
- indices 8,9 and 10,11 ALWAYS unfilled! <--- decision



# Therefore Simplify Hamiltonian

In [7]:
ansatz_obj = BK_Qubit_Reduction(QubitHamiltonian,
                             Hamilt.molecule.n_electrons,
                             Hamilt.molecule.n_qubits)

The H2O ground state under JW is:

$$|\psi \rangle_{HF}^{ground} = | 1 1 1 1 \:  0 0 0 0 0 0 0 0 \rangle$$

and under BK is:

$$|\psi \rangle_{HF}^{ground} = | 1 0 1 0 0 0 0 0 0 0 0 0 \rangle$$


we have **fixed occupations** of indices [0,1,8,9,10,11]

- therefore can remove these terms from the Hamiltonian!

In [8]:
reduced_Qubit_Hamiltonian = ansatz_obj.Remove_indices_from_Hamiltonian(removed_indices)
print('Hamiltonian size reduced: {} --> {}'.format(len(list(QubitHamiltonian)), len(list(reduced_Qubit_Hamiltonian))))

Hamiltonian size reduced: 1388 --> 1100


In [9]:
qubit_re_label_dict, reduced_RE_LABELLED_Qubit_Hamiltonian = ansatz_obj.Re_label_Hamiltonian(reduced_Qubit_Hamiltonian)
reduced_RE_LABELLED_Qubit_Hamiltonian

(-46.74642527808354+0j) [] +
(0.00546090006557411+0j) [X0] +
(0.10688883191080306+0j) [X0 X1 X2] +
(0.0024331089238630566+0j) [X0 X1 X2 X3] +
(0.0018823069706634735+0j) [X0 X1 X2 X3 Y9] +
(-0.003543941201579209+0j) [X0 X1 X2 Z3] +
(-0.0014795610736064787+0j) [X0 X1 Z2 X3 Z7 Y8 X9] +
(0.0015611859902872525+0j) [X0 X1 Z2 Y3 Z5] +
(-0.006060933070820538+0j) [X0 X1 X3] +
(-0.000851323751960051+0j) [X0 X1 X3 X4] +
(0.0010022198919638747+0j) [X0 X1 X3 X4 Y5 Z9] +
(-0.0007985318024164989+0j) [X0 X1 X3 X4 Y9] +
(0.005685081540479361+0j) [X0 X1 X3 Y4 Y5 Z9] +
(0.0007985318024164989+0j) [X0 X1 X3 Y4 Z5 Z7 Z8 X9] +
(-0.0002282715860113743+0j) [X0 X1 X3 Z4 Y5 Z9] +
(-9.109928253980847e-05+0j) [X0 X1 X3 Z4 Z5 Z7 Y8 X9] +
(0.0007074325198766904+0j) [X0 X1 X3 Z4 Z7 Y8 X9] +
(0.0010022198919638747+0j) [X0 X1 X3 X5 Z9] +
(-0.0033242974434877907+0j) [X0 X1 X3 X6] +
(-0.0038893419640023737+0j) [X0 X1 X3 X6 Y9] +
(0.0038893419640023737+0j) [X0 X1 X3 Y6 Z8 X9] +
(0.005356836796490536+0j) [X0 X1 X3 Z6 Z7 Y8

In [10]:
new_Molecular_H_MATRIX = Hamilt.Get_sparse_Qubit_Hamiltonian_matrix(reduced_RE_LABELLED_Qubit_Hamiltonian)


from scipy.sparse.linalg import eigs
eig_values, eig_vectors = eigs(new_Molecular_H_MATRIX)
new_FCI_Energy = min(eig_values)


print('new_FCI = ', new_FCI_Energy, 'VS old FCI:', Hamilt.molecule.fci_energy)

new_FCI =  (-75.81619891022201-1.1502236471931052e-08j) VS old FCI: -75.73161740068062


## Looking at Ansatz

In [11]:
Qubit_Op_list_Second_Quant_CC_Ops_ia, Qubit_Op_list_Second_Quant_CC_Ops_ijab = ansatz_obj.UCCSD_single_trotter_step(reduced_Sec_Quant_CC_ops_ia,
                                                                                                                    reduced_Sec_Quant_CC_ops_ijab,
                                                                                                                   transformation='BK')

In [12]:
ia_terms, ijab_terms, ia_theta, ijab_theta = ansatz_obj.Get_ia_and_ijab_terms()
print('REDUCTION')
print('ia_terms', len(ia_terms), 'TO', len(reduced_Sec_Quant_CC_ops_ia))
print('ijab_terms', len(ijab_terms), 'TO', len(reduced_Sec_Quant_CC_ops_ijab))

REDUCTION
ia_terms 20 TO 8
ijab_terms 65 TO 10


In [13]:
reduced_CC_ijab = ansatz_obj.Remove_indices_from_CC_qubit_operators(Qubit_Op_list_Second_Quant_CC_Ops_ijab, removed_indices)
len(reduced_CC_ijab)

10

In [14]:
reduced_RE_LABELLED_CC_ijab = ansatz_obj.Re_label_CC_qubit_operators( qubit_re_label_dict, 
                                                                   reduced_CC_ijab)
#reduced_RE_LABELLED_CC_ijab

In [15]:
reduced_CC_ia = ansatz_obj.Remove_indices_from_CC_qubit_operators(Qubit_Op_list_Second_Quant_CC_Ops_ia, removed_indices)

reduced_RE_LABELLED_CC_ia = ansatz_obj.Re_label_CC_qubit_operators( qubit_re_label_dict, 
                                                                   reduced_CC_ia)
#reduced_RE_LABELLED_CC_ia

## HF input state

In [16]:
print('old input = ', ansatz_obj.Get_BK_HF_state_in_OCC_basis())

print('BUT following indices removed:', removed_indices)

ansatz_obj.New_BK_HF_state(removed_indices)

old input =  [1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0.]
BUT following indices removed: [12, 13, 6, 7]


array([1., 0., 1., 0., 1., 0., 1., 0., 0., 0.])

# Ansatz

look at: From PHYS. REV. X, **8**, 031022 (2018)

can make a further reduction!

In [17]:
import random
theta_ia_random_input = [random.uniform(0, 2*np.pi) for _ in range(len(reduced_RE_LABELLED_CC_ia))]
theta_ijab_random_input = [random.uniform(0, 2*np.pi) for _ in range(len(reduced_RE_LABELLED_CC_ijab))]

theta_combined_random_input = [*theta_ia_random_input, *theta_ijab_random_input]

In [18]:
input_state = ansatz_obj.New_BK_HF_state(removed_indices)
print('input_state =', input_state)

UCCSD_ansatz_Q_Circ_obj = Ansatz_Circuit(input_state,
                                     reduced_RE_LABELLED_CC_ia, 
                                     reduced_RE_LABELLED_CC_ijab)

ansatz_cirq_circuit =UCCSD_ansatz_Q_Circ_obj.Get_Full_HF_UCCSD_QC(
                                        Theta_param_list_ia=theta_ia_random_input, 
                                         Theta_param_list_ijab=theta_ijab_random_input,
                                         ia_first=True)
ansatz_cirq_circuit

input_state = [1. 0. 1. 0. 1. 0. 1. 0. 0. 0.]


┌──┐                 ┌───────────┐                 ┌──────────┐                ┌─────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     ┌──┐
0: ───X───H───@────────────────────────────────────────────────@───H───Rx(0.5π)───@─────────────────────────────────────────────────@───Rx(-0.5π)─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────@────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────@───H───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────@────────────────────────────@────H─────────────H────────────@─────────────────────────────────@───────────H───Rx(0.5π)───@────────────────@───Rx(-0.5π)───Rx(0.5π)───@────────────────────────@───Rx(-0.5π)───Rx(0.5π)───@────────────────────────────────@───Rx(-0.5π)───Rx(0.5π)───@────────────────────────────────────────@──────────Rx(-0.5π)───H───@─────────────────────────────────@───────────H──────────H───@─────────────────────────────────────────────────@──────────H──────────Rx(0.5π)───@──────────────────

# Running Example

# Using measurement reduction!

In [19]:
from quchem.LCU_method import *

In [20]:
Hamiltonian_graph_obj = Openfermion_Hamiltonian_Graph(reduced_RE_LABELLED_Qubit_Hamiltonian)

commutativity_flag = 'AC' ## <- defines relationship between sets!!!
plot_graph = False
Graph_colouring_strategy='largest_first'
anti_commuting_sets = Hamiltonian_graph_obj.Get_Clique_Cover_as_QubitOp(commutativity_flag, Graph_colouring_strategy=Graph_colouring_strategy, plot_graph=plot_graph)
anti_commuting_sets

Building Graph Edges: 100%|##########| 1100/1100 [00:53<00:00, 20.68it/s] 


{0: [(-46.74642527808354+0j) []],
 1: [(0.2201657104821654+0j) [Z7]],
 2: [(1.4984847642467822+0j) [Z6 Z7],
  (0.0060913406004651375+0j) [Y6],
  (0.019879604731802156+0j) [X6 Z7]],
 3: [(1.5122730283781192+0j) [Z6],
  (0.0060913406004651375+0j) [X6],
  (-0.00769692353087188+0j) [Y6 Z7]],
 4: [(12.731832323407453+0j) [Z0 Z1],
  (0.13315875677935532+0j) [X1 Z2],
  (-0.0017857392100000093+0j) [Y0 Z1],
  (0.012707539341148228+0j) [X0 Z1],
  (-7.187277876848467e-08+0j) [Y1 Y3 Z4 X5],
  (1.011714733168724e-05+0j) [X1 Y2],
  (-0.009249320846609038+0j) [X1 X2],
  (-0.07558792883842516+0j) [Y1 X3 Z7 Z8 X9],
  (-0.05166504072997431+0j) [Y1 Y3 Z5],
  (0.002541551585858021+0j) [Y1 X3 Z7 Y8 X9],
  (0.0034691314127466223+0j) [Y1 X3 Z7 X8 X9],
  (3.302572040151919e-07+0j) [Y1 X3 Z9]],
 5: [(12.739078962683026+0j) [Z0],
  (-0.1285290377823849+0j) [Y0 Y1 X2],
  (-7.187277876848467e-08+0j) [Y0 X1 Y3 X4 X5],
  (0.00546090006557411+0j) [X0],
  (-1.0117147331686373e-05+0j) [Y0 X1 Z3],
  (-0.004619601849638

In [21]:
input_state = ansatz_obj.New_BK_HF_state(removed_indices)
print('input_state =', input_state)

UCCSD_ansatz_Q_Circ_obj = Ansatz_Circuit(input_state,
                                     reduced_RE_LABELLED_CC_ia, 
                                     reduced_RE_LABELLED_CC_ijab)

ansatz_cirq_circuit =UCCSD_ansatz_Q_Circ_obj.Get_Full_HF_UCCSD_QC(Theta_param_list_ia=theta_ia_random_input, 
                                         Theta_param_list_ijab=theta_ijab_random_input,
                                         ia_first=True)
ansatz_cirq_circuit

input_state = [1. 0. 1. 0. 1. 0. 1. 0. 0. 0.]


┌──┐                 ┌───────────┐                 ┌──────────┐                ┌─────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     ┌──┐
0: ───X───H───@────────────────────────────────────────────────@───H───Rx(0.5π)───@─────────────────────────────────────────────────@───Rx(-0.5π)─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────@────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────@───H───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────@────────────────────────────@────H─────────────H────────────@─────────────────────────────────@───────────H───Rx(0.5π)───@────────────────@───Rx(-0.5π)───Rx(0.5π)───@────────────────────────@───Rx(-0.5π)───Rx(0.5π)───@────────────────────────────────@───Rx(-0.5π)───Rx(0.5π)───@────────────────────────────────────────@──────────Rx(-0.5π)───H───@─────────────────────────────────@───────────H──────────H───@─────────────────────────────────────────────────@──────────H──────────Rx(0.5π)───@──────────────────

In [22]:
N_QUBITS = Hamilt.molecule.n_qubits - len(removed_indices)

def GIVE_ENERGY_lin_alg(theta_ia_ijab):
    
    theta_ia= theta_ia_ijab[:(len(reduced_theta_parameters_ia))]
    theta_ijab=theta_ia_ijab[(len(reduced_theta_parameters_ia)):]
   
    ansatz_cirq_circuit =UCCSD_ansatz_Q_Circ_obj.Get_Full_HF_UCCSD_QC(
                                        Theta_param_list_ia=theta_ia, 
                                         Theta_param_list_ijab=theta_ijab,
                                         ia_first=True)

    VQE_exp_LCU_lin_alg = VQE_Experiment_LCU_UP_lin_alg(anti_commuting_sets,
                 ansatz_cirq_circuit,
                 N_QUBITS,                     # <--- NOTE THIS
                 N_indices_dict=None) 
    
    return VQE_exp_LCU_lin_alg.Calc_Energy().real

In [23]:
import random
theta_ia_random_input = [random.uniform(0, 2*np.pi) for _ in range(len(reduced_RE_LABELLED_CC_ia))]
theta_ijab_random_input = [random.uniform(0, 2*np.pi) for _ in range(len(reduced_RE_LABELLED_CC_ijab))]

theta_combined_random_input = [*theta_ia_random_input, *theta_ijab_random_input]

GIVE_ENERGY_lin_alg(theta_combined_random_input)

### COMPARE  to linalg

In [24]:
N_QUBITS = Hamilt.molecule.n_qubits - len(removed_indices)

lin_alg = Ansatz_lin_alg(N_QUBITS)

input_state = ansatz_obj.New_BK_HF_state(removed_indices)
print('input_state =', input_state)

HF_ket = lin_alg.Get_reference_ket(input_state)
print(HF_ket.shape)

input_state = [1. 0. 1. 0. 1. 0. 1. 0. 0. 0.]
(1024, 1)


In [25]:
theta_ia= theta_combined_random_input[:(len(reduced_theta_parameters_ia))]
theta_ijab=theta_combined_random_input[(len(reduced_theta_parameters_ia)):]

# UCC_ia_operator = lin_alg.Get_ijab_UCC_matrix_NO_TROT(theta_ia, reduced_RE_LABELLED_CC_ia)
UCC_ia_operator = lin_alg.Get_ia_UCC_matrix_WITH_trot_SINGLE_STEP(theta_ia, reduced_RE_LABELLED_CC_ia)

# UCC_ijab_operator = lin_alg.Get_ia_UCC_matrix_NO_TROT(theta_ijab, reduced_RE_LABELLED_CC_ijab)
UCC_ijab_operator = lin_alg.Get_ijab_UCC_matrix_WITH_trot_SINGLE_STEP(theta_ijab, reduced_RE_LABELLED_CC_ijab)

Qubit_H_matrix = lin_alg.Get_Qubit_Hamiltonian_matrix(reduced_RE_LABELLED_Qubit_Hamiltonian)

In [26]:
ket_post_single_excitation = UCC_ia_operator.dot(HF_ket)
ket_post_double_excitation = UCC_ijab_operator.dot(ket_post_single_excitation)
energy = lin_alg.Calc_energy_of_state(ket_post_double_excitation, Qubit_H_matrix)
print(energy)

(-19.82144568443575-8.673617379884035e-19j)


In [27]:
N_QUBITS = Hamilt.molecule.n_qubits - len(removed_indices)

def lin_alg_energy(theta_ia_ijab):
    
    lin_alg = Ansatz_lin_alg(N_QUBITS)

    
    HF_ket = lin_alg.Get_reference_ket(input_state)
    
    theta_ia= theta_ia_ijab[:(len(reduced_theta_parameters_ia))]
    theta_ijab=theta_ia_ijab[(len(reduced_theta_parameters_ia)):]
    
    
#     UCC_ia_operator = lin_alg.Get_ijab_UCC_matrix_NO_TROT(theta_ia, reduced_RE_LABELLED_CC_ia)
    UCC_ia_operator = lin_alg.Get_ia_UCC_matrix_WITH_trot_SINGLE_STEP(theta_ia, reduced_RE_LABELLED_CC_ia)

#     UCC_ijab_operator = lin_alg.Get_ia_UCC_matrix_NO_TROT(theta_ijab, reduced_RE_LABELLED_CC_ijab)
    UCC_ijab_operator = lin_alg.Get_ijab_UCC_matrix_WITH_trot_SINGLE_STEP(theta_ijab, reduced_RE_LABELLED_CC_ijab)

    
    ket_post_single_excitation = UCC_ia_operator.dot(HF_ket)
    ket_post_double_excitation = UCC_ijab_operator.dot(ket_post_single_excitation)
    energy = lin_alg.Calc_energy_of_state(ket_post_double_excitation, Qubit_H_matrix)
    return energy

In [28]:
lin_alg_energy(theta_combined_random_input)

(-19.82144568443575-8.673617379884035e-19j)

# Optimization

In [29]:
from quchem.Scipy_Optimizer import *

In [ ]:
# Nelder-Mead

GG = Optimizer(GIVE_ENERGY_lin_alg,
                   theta_combined_random_input,
                   args=(),
                   method='Nelder-Mead',
                   jac=None,
                   hess=None,
                   hessp=None,
                      bounds=None,
                   constraints=None,
                   display_convergence_message=True,
                   display_steps=True)

GG.get_env(50)
GG.plot_convergence()
plt.show()

In [31]:
# def calc_gradient_ADAM(theta_ia_theta_jab_list):
    
#     grad_list=[]
    
#     for index, theta in enumerate(theta_ia_theta_jab_list):
        
#         new_theta_list = theta_ia_theta_jab_list.copy()
#         new_theta_list[index] = theta + np.pi/4
        
#         Obs_PLUS = GIVE_ENERGY_lin_alg(new_theta_list)
               
        
#         new_theta_list[index] = theta - np.pi/4
#         Obs_MINUS = GIVE_ENERGY_lin_alg(new_theta_list)
        
#         gradient = Obs_PLUS - Obs_MINUS
        
#         grad_list.append(gradient)
        

#     return np.array(grad_list)

In [32]:
### optimizer
from quchem.Scipy_Optimizer import _minimize_Adam, Optimizer

In [33]:
# custom_optimizer_DICT = {'learning_rate': 0.1, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-8,
#                            'delta': 1e-8, 'maxfev': 15000}

# GG = Optimizer(GIVE_ENERGY_lin_alg,
#                    theta_combined_random_input,
#                    args=(),
#                    method=_minimize_Adam,
#                    jac=calc_gradient_ADAM,
#                    hess=None,
#                    hessp=None,
#                       bounds=None,
#                    constraints=None,
#                    tol=1e-20, 
#                    display_convergence_message=True,
#                    display_steps=True, 
#                    custom_optimizer_DICT=custom_optimizer_DICT)
# GG.get_env(100)
# GG.plot_convergence()
# plt.show()

In [34]:
Hamilt.molecule.fci_energy

-75.73161740068062

# Linear Alg CHECK!